In [139]:
import selenium
from selenium.webdriver.common.by import By
from selenium import webdriver
import time

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
from konlpy.tag import Okt
from PIL import Image
import numpy as np
import os, sys

In [140]:
driver = webdriver.Chrome()
url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver'
driver.get(url)

In [141]:
result = {}
list_ranking = driver.find_element(By.CLASS_NAME, 'list_ranking')
tds = list_ranking.find_elements(By.CLASS_NAME, 'tit3')

In [142]:
# 영화 제목, 링크 딕셔너리 저장
for td in tds:
    a = td.find_element(By.TAG_NAME, 'a')
    href = a.get_attribute('href')
    title = a.get_attribute('title')
    result[title] = href

In [143]:
# 영화 리스트에 있는 링크들 하나하나 열기
# for key in result.keys():
#     link = result[key]
#     driver.get(link)
#     time.sleep(1)

In [144]:
# 영화 올빼미 창 열기
# driver.get(result['올빼미'])

In [145]:
# 영화 1위 창 열기
first_movie = list(result.values())[0]
driver.get(first_movie)

In [146]:
# 평점 창 열기
driver.find_element(By.XPATH, '//*[@id="movieEndTabMenu"]/li[5]').click()

In [147]:
# 리뷰 iframe 창 열기
iframe = driver.find_element(By.ID,'pointAfterListIframe')
frame_src = iframe.get_attribute('src')
driver.get(frame_src)

In [148]:
# 스포일러 체크가 안되어 있을 경우 스포일러 보기 체크.
spoiler_YN = driver.find_element(By.ID, 'spoilerYnCheckBox')
if spoiler_YN.get_attribute('class') == 'blind ':
    driver.find_element(By.ID, 'spoilerYnLable').click()

In [149]:
# 댓글 최신순 클릭
newst = driver.find_element(By.XPATH, '//*[@id="orderCheckbox"]/ul[1]/li[2]/a').click()

In [150]:
current_url = driver.current_url

In [151]:
reple_list = []
max_page = 100

for i in range(1,max_page):
    ems = driver.find_elements(By.TAG_NAME, 'em')  
    ems[-1].text
    if ems[-1].text == "다음":
        driver.get(current_url + f'&page={i}')
        # 댓글 가져와 리스트에 넣기
        reples = driver.find_elements(By.CLASS_NAME, 'score_reple')
        for reple in reples:
            reple_spans = reple.find_elements(By.TAG_NAME, 'span')
            # span 클래스명이 ico_viewer 일 경우 제외
            if reple_spans[0].get_attribute('class') == 'ico_viewer':
                reple_list.append(reple_spans[1].text)
            else:
                reple_list.append(reple_spans[0].text)


In [ ]:
reple_text = ' '.join(reple_list)
reple_list

In [158]:
with open('reple.txt', 'w') as f:
    f.write(reple_text)

In [161]:
okt = Okt()
nouns = okt.nouns(reple_text)
nouns

['갑자기',
 '큰아들',
 '죽',
 '눈물',
 '강요',
 '헐리우드',
 '신파',
 '공감',
 '아바타',
 '물의',
 '호흡',
 '자알',
 '편',
 '능',
 '나와라',
 '스토리',
 '신경',
 '사람',
 '라면',
 '죽',
 '스토리',
 '그래픽',
 '물의',
 '족속',
 '어디',
 '감',
 '설리',
 '가족',
 '첫째',
 '아들',
 '죽',
 '때',
 '눈물',
 '다른',
 '애',
 '구',
 '가야',
 '아들',
 '가야',
 '엄마',
 '심정',
 '제',
 '이해',
 '그',
 '감정',
 '순',
 '감동',
 '이',
 '외',
 '편',
 '그래픽',
 '움',
 '단계',
 '진보',
 '발전',
 '동안',
 '영화',
 '한편',
 '게임',
 '연구',
 '작품',
 '이',
 '정도',
 '상미',
 '스토리',
 '가장',
 '테이',
 '얌',
 '하차',
 '줄',
 '진짜',
 '감독',
 '해명',
 '좀',
 '마지막',
 '장면',
 '눈물',
 '블랙',
 '팬서',
 '엔딩',
 '느낌',
 '더',
 '엔딩',
 '씬',
 '내용',
 '악당',
 '제일',
 '사임',
 '스포',
 '영화',
 '사람',
 '역시',
 '아바타',
 '상미',
 '스토리',
 '얼척',
 '편',
 '활약',
 '부족',
 '주인공',
 '적장',
 '애기',
 '공격',
 '족장',
 '잠적',
 '연기',
 '스토리',
 '감동',
 '올해',
 '가장',
 '크게',
 '울음',
 '인간',
 '욕심',
 '멧',
 '케이',
 '족',
 '바디',
 '고래사냥',
 '포경',
 '볼',
 '때',
 '죄책감',
 '듭니다네테이얌',
 '왜',
 '마블',
 '삼',
 '진짜',
 '연출',
 '영상',
 '미가',
 '외계행성',
 '다큐',
 '라면',
 '점',
 '모자라',
 '스토리',
 '배경',
 '속',
 '서로',
 '가족영화',
 '존잼',
 '글자',


In [155]:
for i in nouns:
    if i == '상미':
        nouns.remove(i)
        nouns.append('영상미')

In [156]:
words = [n for n in nouns if len(n) > 1]


c = Counter(words)
c


Counter({'갑자기': 21,
         '큰아들': 5,
         '눈물': 48,
         '강요': 1,
         '헐리우드': 3,
         '신파': 4,
         '공감': 7,
         '아바타': 149,
         '물의': 33,
         '호흡': 1,
         '자알': 1,
         '나와라': 1,
         '스토리': 216,
         '신경': 3,
         '사람': 40,
         '라면': 7,
         '그래픽': 27,
         '족속': 1,
         '어디': 23,
         '설리': 58,
         '가족': 141,
         '첫째': 12,
         '아들': 62,
         '다른': 20,
         '가야': 4,
         '엄마': 17,
         '심정': 1,
         '이해': 19,
         '감정': 10,
         '감동': 69,
         '단계': 3,
         '진보': 1,
         '발전': 7,
         '동안': 14,
         '영화': 229,
         '한편': 5,
         '게임': 3,
         '연구': 1,
         '작품': 9,
         '정도': 30,
         '가장': 9,
         '테이': 21,
         '하차': 1,
         '진짜': 85,
         '감독': 29,
         '해명': 1,
         '마지막': 78,
         '장면': 79,
         '블랙': 2,
         '팬서': 1,
         '엔딩': 6,
         '느낌': 69,
         '내용': 42,
      